In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import (
    LinearRegression,Lasso, Ridge, RidgeCV, LassoCV, LogisticRegression
)
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import warnings

warnings.filterwarnings('ignore')

In [2]:
RANDOM_STATE = 42

In [3]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [4]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
model_linear = LinearRegression().fit(X_train, y_train)
model_linear.score(X_test, y_test)

0.6687594935356318

In [6]:
model_ridge = Ridge().fit(X_train, y_train)
model_ridge.score(X_test, y_test)

0.6662221670168519

In [7]:
model_lasso = Lasso().fit(X_train, y_train)
model_lasso.score(X_test, y_test)

0.6671453631686304

3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [8]:
parameters = [
    {'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000]}
]

### GridSearchCV

In [9]:
estimator_ridge = GridSearchCV(Ridge(), parameters)
estimator_ridge.fit(X_train, y_train)
estimator_ridge.best_params_

{'alpha': 1e-05}

In [10]:
estimator_ridge.score(X_test, y_test)
# R2 увеличился

0.6687594856409733

In [11]:
estimator_lasso = GridSearchCV(Lasso(), parameters)
estimator_lasso.fit(X_train, y_train)
estimator_lasso.best_params_

{'alpha': 1e-05}

In [12]:
estimator_lasso.score(X_test, y_test)
# R2 увеличился (лучше)

0.6687598638315155

### RidgeCV

In [13]:
model_ridgecv = RidgeCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000))
model_ridgecv.fit(X_train, y_train)
model_ridgecv.score(X_test, y_test)
# R2 увеличился (лучше)

0.6687510090373743

### LassoCV

In [14]:
model_lassocv = LassoCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000))
model_lassocv.fit(X_train, y_train)
model_lassocv.score(X_test, y_test)
# R2 увеличился (лучше)

0.6687598638315155

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [15]:
pipe_ridge_standard_scaler = Pipeline(
    [('scaler', StandardScaler()), ('model', Ridge())]
)
pipe_ridge_standard_scaler.fit(X_train, y_train)
pipe_ridge_standard_scaler.score(X_test, y_test)
# R2, по сравнению с Ridge(), увеличился (лучше)

0.6684624359643561

In [16]:
pipe_lasso_standard_scaler = Pipeline(
    [('scaler', StandardScaler()), ('model', Lasso())]
)
pipe_lasso_standard_scaler.fit(X_train, y_train)
pipe_lasso_standard_scaler.score(X_test, y_test)
# R2, по сравнению с Lasso(), заметно уменьшился

0.6239428734251422

In [17]:
pipe_ridge_minmax_scaler = Pipeline(
    [('scaler', MinMaxScaler()), ('model', Ridge())]
)
pipe_ridge_minmax_scaler.fit(X_train, y_train)
pipe_ridge_minmax_scaler.score(X_test, y_test)
# R2 больше чем у Ridge(), но меньше чем с StandardScaler()

0.6764100365423598

In [18]:
pipe_lasso_minmax_scaler = Pipeline(
    [('scaler', MinMaxScaler()), ('model', Lasso())]
)
pipe_lasso_minmax_scaler.fit(X_train, y_train)
pipe_lasso_minmax_scaler.score(X_test, y_test)
# R2 значительное уменьшение по сравнению как с чистым Lasso(), так и с StandardScaler() 

0.2573921442545195

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [19]:
pipe_ridgecv_standard_scaler = Pipeline(
    [('scaler', StandardScaler()), ('model', RidgeCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_ridgecv_standard_scaler.fit(X_train, y_train)
pipe_ridgecv_standard_scaler.score(X_test, y_test)
# R2, по сравнению с чистым RidgeCV(), уменьшился

0.6659677905050798

In [20]:
pipe_lassocv_standard_scaler = Pipeline(
    [('scaler', StandardScaler()), ('model', LassoCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_lassocv_standard_scaler.fit(X_train, y_train)
pipe_lassocv_standard_scaler.score(X_test, y_test)
# R2, по сравнению с чистым LassoCV(), немного уменьшился

0.668759038334717

In [21]:
pipe_ridgecv_minmax_scaler = Pipeline(
    [('scaler', MinMaxScaler()), ('model', RidgeCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_ridgecv_minmax_scaler.fit(X_train, y_train)
pipe_ridgecv_minmax_scaler.score(X_test, y_test)
# R2, по сравнению как с чистым RidgeCV(), так и с StandardScaler() заметно увеличился

0.6700309977617445

In [22]:
pipe_lassocv_minmax_scaler = Pipeline(
    [('scaler', MinMaxScaler()), ('model', LassoCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_lassocv_minmax_scaler.fit(X_train, y_train)
pipe_lassocv_minmax_scaler.score(X_test, y_test)
# R2, по сравнению как с чистым LassoCV(), так и с StandardScaler() немного увеличился

0.6687605073677362

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [23]:
pipe_ridge_standard_scaler_polynomial = Pipeline(
    [('scaler', StandardScaler()), ('polynomial', PolynomialFeatures(2)), ('model', Ridge())]
)
pipe_ridge_standard_scaler_polynomial.fit(X_train, y_train)
pipe_ridge_standard_scaler_polynomial.score(X_test, y_test)
# R2 значительное увеличение по сравнению с предыдущими Ridge

0.816294899023

In [24]:
pipe_lasso_standard_scaler_polynomial = Pipeline(
    [('scaler', StandardScaler()), ('polynomial', PolynomialFeatures(2)), ('model', Lasso())]
)
pipe_lasso_standard_scaler_polynomial.fit(X_train, y_train)
pipe_lasso_standard_scaler_polynomial.score(X_test, y_test)
# R2 значительное увеличение по сравнению с предыдущими Lasso

0.7322762586137164

In [25]:
pipe_ridge_minmax_scaler_polynomial = Pipeline(
    [('scaler', MinMaxScaler()), ('polynomial', PolynomialFeatures(2)), ('model', Ridge())]
)
pipe_ridge_minmax_scaler_polynomial.fit(X_train, y_train)
pipe_ridge_minmax_scaler_polynomial.score(X_test, y_test)
# R2 увеличение по сравнению с предыдущими Ridge

0.8299337208033138

In [26]:
pipe_lasso_minmax_scaler_polynomial = Pipeline(
    [('scaler', MinMaxScaler()), ('polynomial', PolynomialFeatures(2)), ('model', Lasso())]
)
pipe_lasso_minmax_scaler_polynomial.fit(X_train, y_train)
pipe_lasso_minmax_scaler_polynomial.score(X_test, y_test)
# R2 значительное уменьшение по сравнению как с чистым Lasso(), так и с StandardScaler() 

0.2611262741735658

In [27]:
pipe_ridgecv_standard_scaler_polynomial = Pipeline(
    [('scaler', StandardScaler()),
     ('polynomial', PolynomialFeatures(2)),
     ('model', RidgeCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_ridgecv_standard_scaler_polynomial.fit(X_train, y_train)
pipe_ridgecv_standard_scaler_polynomial.score(X_test, y_test)
# R2 значительное увеличение по сравнению с предыдущими RidgeCV()

0.8180465877243803

In [28]:
pipe_lassocv_standard_scaler_polynomial = Pipeline(
    [('scaler', StandardScaler()),
     ('polynomial', PolynomialFeatures(2)),
     ('model', LassoCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_lassocv_standard_scaler_polynomial.fit(X_train, y_train)
pipe_lassocv_standard_scaler_polynomial.score(X_test, y_test)
# R2 значительное увеличение по сравнению с предыдущими LassoCV()

0.8122168137984359

In [29]:
pipe_ridgecv_minmax_scaler_polynomial = Pipeline(
    [('scaler', MinMaxScaler()),
     ('polynomial', PolynomialFeatures(2)),
     ('model', RidgeCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_ridgecv_minmax_scaler_polynomial.fit(X_train, y_train)
pipe_ridgecv_minmax_scaler_polynomial.score(X_test, y_test)
# R2 значительное увеличение по сравнению с предыдущими RidgeCV()

0.8500630422288056

In [30]:
pipe_lassocv_minmax_scaler_polynomial = Pipeline(
    [('scaler', MinMaxScaler()),
     ('polynomial', PolynomialFeatures(2)),
     ('model', LassoCV(alphas=(1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)))]
)
pipe_lassocv_minmax_scaler_polynomial.fit(X_train, y_train)
pipe_lassocv_minmax_scaler_polynomial.score(X_test, y_test)
# R2 значительное увеличение по сравнению с предыдущими LassoCV()

0.8390581680518306

In [31]:
pipe_linear = Pipeline ( [('scaler', MinMaxScaler()), ('polynomial', PolynomialFeatures(2)), ('model', LinearRegression())])
pipe_linear.fit(X_train, y_train)
pipe_linear.score(X_test, y_test)
# для себя

0.8033778680650641

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

In [32]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('polynomial', PolynomialFeatures()),
    ('model', LassoCV())
])
parameters = [
    {
        'scaler': [StandardScaler(), MinMaxScaler()],
        'polynomial__degree': (0, 1, 2, 3, 4, 5, 6),
        'model': [LassoCV(), RidgeCV()],
        'model__alphas': (1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100, 1_000, 10_000, 10_000)
    },
]
grid_search = GridSearchCV(pipeline, parameters)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)
print(grid_search.score(X_test, y_test))

# R2 выше всех предыдущих

{'model': RidgeCV(alphas=1), 'model__alphas': 1, 'polynomial__degree': 5, 'scaler': MinMaxScaler()}
0.8593048637026933


http://archive.ics.uci.edu/ml/datasets/Adult

In [33]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [34]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [35]:
data.columns = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education_num',
    'marital_status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital_gain',
    'capital_loss',
    'hours_per_week',
    'native_country',
    'income'
]
data.loc[data.income == '<=50K', 'income_over_50k'] = 0
data.loc[data.income == '>50K', 'income_over_50k'] = 1
data.income_over_50k = data.income_over_50k.astype(int)
data = data.drop('income', axis=1)
data.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income_over_50k
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [36]:
y = data.income_over_50k
X = data.drop('income_over_50k', axis=1)

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [37]:
print(data.isnull().sum())
# пропусков нет

age                0
workclass          0
fnlwgt             0
education          0
education_num      0
marital_status     0
occupation         0
relationship       0
race               0
sex                0
capital_gain       0
capital_loss       0
hours_per_week     0
native_country     0
income_over_50k    0
dtype: int64


10. Выберите колонки с числовыми и категориальными переменными.

In [38]:
columns = X.columns
X_num_cols = X._get_numeric_data().columns
X_cat_cols = list(set(columns).difference(set(X_num_cols)))

In [39]:
X_num_cols = X_num_cols.to_list()
X_num_cols

['age',
 'fnlwgt',
 'education_num',
 'capital_gain',
 'capital_loss',
 'hours_per_week']

In [40]:
X_cat_cols

['marital_status',
 'sex',
 'relationship',
 'race',
 'occupation',
 'native_country',
 'education',
 'workclass']

11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [41]:
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

numeric_transformer = MinMaxScaler()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, X_num_cols),
        ("cat", categorical_transformer, X_cat_cols),
    ]
)

clf = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [42]:
y.value_counts()

0    37155
1    11687
Name: income_over_50k, dtype: int64

In [43]:
print(f'Accuracy: {accuracy_score(y, np.array([0 for _ in range(len(y))]))}')
print(f'F1-score: {f1_score(y, np.array([0 for _ in range(len(y))]))}')

Accuracy: 0.7607182343065395
F1-score: 0.0


13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [44]:
print(f'Accuracy: {accuracy_score(y, np.array([0 for _ in range(len(y))]))}')
print(f'F1-score: {f1_score(y, np.array([0 for _ in range(len(y))]))}')

Accuracy: 0.7607182343065395
F1-score: 0.0


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [45]:
(X == '?').sum()

age                  0
workclass         2799
fnlwgt               0
education            0
education_num        0
marital_status       0
occupation        2809
relationship         0
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
native_country     857
dtype: int64

In [46]:
data_columns = X.columns
imputer = SimpleImputer(missing_values='?', strategy='most_frequent')
new_data = pd.DataFrame(imputer.fit_transform(X), columns=data_columns)
(new_data== '?').sum()

age               0
workclass         0
fnlwgt            0
education         0
education_num     0
marital_status    0
occupation        0
relationship      0
race              0
sex               0
capital_gain      0
capital_loss      0
hours_per_week    0
native_country    0
dtype: int64

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

In [47]:
clf_lg = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

print('LogisticRegression')
print('accuracy', cross_val_score(clf_lg, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lg, X, y, cv=5, scoring = 'f1').mean())



print()
print('-----------------------------------------')
print()

clf_lsvc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LinearSVC())]
)
print('LinearSVC')
print('accuracy', cross_val_score(clf_lsvc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lsvc, X, y, cv=5, scoring = 'f1').mean())

print()
print('-----------------------------------------')
print()

clf_svc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", SVC())]
)
print('SVC')
print('accuracy', cross_val_score(clf_svc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_svc, X, y, cv=5, scoring = 'f1').mean())

# Результат стал незначительно хуже

LogisticRegression
accuracy 0.8510709028853158
f1 0.6563242792121816

-----------------------------------------

LinearSVC
accuracy 0.8529135478362626
f1 0.6578163598233313

-----------------------------------------

SVC
accuracy 0.8399739469040318
f1 0.6200828429738441


16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [48]:
data_clear = data.replace('?', np.nan).dropna()

In [49]:
y = data_clear.income_over_50k
X = data_clear.drop('income_over_50k', axis=1)

clf_lg = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LogisticRegression())]
)

print('LogisticRegression')
print('accuracy', cross_val_score(clf_lg, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lg, X, y, cv=5, scoring = 'f1').mean())



print()
print('-----------------------------------------')
print()

clf_lsvc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", LinearSVC())]
)
print('LinearSVC')
print('accuracy', cross_val_score(clf_lsvc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_lsvc, X, y, cv=5, scoring = 'f1').mean())

print()
print('-----------------------------------------')
print()

clf_svc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", SVC())]
)
print('SVC')
print('accuracy', cross_val_score(clf_svc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_svc, X, y, cv=5, scoring = 'f1').mean())

LogisticRegression
accuracy 0.8468445746108516
f1 0.6601244618889603

-----------------------------------------

LinearSVC
accuracy 0.8485030154158197
f1 0.6616132960278641

-----------------------------------------

SVC
accuracy 0.8356995943179577
f1 0.6267120053492474


 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [50]:
clf_svc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", GradientBoostingClassifier())]
)
print('GradientBoostingClassifier')
print('accuracy', cross_val_score(clf_svc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_svc, X, y, cv=5, scoring = 'f1').mean())

print()
print('-----------------------------------------')
print()

clf_svc = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestClassifier())]
)
print('RandomForestClassifier')
print('accuracy', cross_val_score(clf_svc, X, y, cv=5, scoring = 'accuracy').mean())
print('f1', cross_val_score(clf_svc, X, y, cv=5, scoring = 'f1').mean())
# GradientBoostingClassifier показал лучший результат

GradientBoostingClassifier
accuracy 0.8629429514670492
f1 0.6869948630650294

-----------------------------------------

RandomForestClassifier
accuracy 0.8488346854845631
f1 0.6694748979062587


18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [51]:
data = pd.read_csv(link, header=None)
data.iloc[:,14] = data.iloc[:,14].replace(['<=50K','>50K'],[0,1])
data_clear = data.replace('?', np.nan).dropna()
X = data_clear.iloc[:,:-1]
y = data_clear.iloc[:,-1]
columns = X.columns
X_num_cols = X._get_numeric_data().columns.to_list()
X_cat_cols = list(set(columns).difference(set(X_num_cols)))

In [52]:
ct_1 = ColumnTransformer([
    ('scale',  MinMaxScaler(), X_num_cols),
    ('onehot', OneHotEncoder(),  X_cat_cols)
])

ct_2 = ColumnTransformer([
    ('scale',  StandardScaler(), X_num_cols),
    ('onehot', OneHotEncoder(),  X_cat_cols)
])

pipe = Pipeline(steps=[
    ('fill_missing', SimpleImputer(missing_values='?', strategy='most_frequent')),
    ('prepare_columns', 'passthrough'),
    ('clf', 'passthrough')]
)

params = [
    {
        'prepare_columns': [ct_1, ct_2],
        'clf': [
            RandomForestClassifier(),
            GradientBoostingClassifier(),
            LogisticRegression(),
            LinearSVC()
        ]
    }
]

search = GridSearchCV(pipe, params, scoring=['accuracy', 'f1'], refit='f1', n_jobs=-1, verbose=4)
search.fit(X, y)
print(search.best_params_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
{'clf': RandomForestClassifier(), 'prepare_columns': ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                 [0, 2, 4, 10, 11, 12]),
                                ('onehot', OneHotEncoder(),
                                 [1, 3, 5, 6, 7, 8, 9, 13])])}


In [53]:
f1_scores = np.array([search.cv_results_[f'split{i}_test_f1'][0] for i in range(5)])
accuracy_scores = np.array([search.cv_results_[f'split{i}_test_accuracy'][0] for i in range(5)])
print(f"F1-score: {np.mean(accuracy_scores[~np.isnan(accuracy_scores)])}")
print(f"Accuracy: {np.mean(f1_scores[~np.isnan(f1_scores)])}")

F1-score: 0.8478632838314692
Accuracy: 0.6693129648157652
